In [15]:
import pandas as pd
carbon_data = pd.read_csv('../data/raw/carbon.csv')

print(carbon_data.head())

         Date  Price   Open   High    Low    Vol. Change %
0  11/22/2024  70.49  70.96  69.01  69.27  39.97K   -1.03%
1  11/21/2024  68.46  70.78  68.45  69.99  40.13K    2.35%
2  11/20/2024  68.02  69.40  67.89  68.38  27.20K    0.37%
3  11/19/2024  69.47  69.83  67.84  68.13  35.33K   -1.99%
4  11/18/2024  67.28  70.03  66.96  69.51  38.34K    2.21%


In [16]:
carbon_data['Date'] = pd.to_datetime(carbon_data['Date'])
carbon_data.rename(columns={'Change %': 'pct_change'}, inplace=True)
carbon_data.rename(columns={'Vol.': 'Vol'}, inplace=True)
for column in carbon_data.columns[1:]:
    if 'pct' in column:
        carbon_data[column] = carbon_data[column].str.replace('%', '').astype(float) / 100
    elif 'Vol' in column:
        carbon_data[column] = carbon_data[column].str.replace('K', '').astype(float) * 1000
    else:
        carbon_data[column] = carbon_data[column].astype(float)
carbon_data.head()

,Date,Price,Open,High,Low,Vol,pct_change
0,2024-11-22,70.49,70.96,69.01,69.27,39970.0,-0.0103
1,2024-11-21,68.46,70.78,68.45,69.99,40130.0,0.0235
2,2024-11-20,68.02,69.40,67.89,68.38,27200.0,0.0037
3,2024-11-19,69.47,69.83,67.84,68.13,35330.0,-0.0199
4,2024-11-18,67.28,70.03,66.96,69.51,38340.0,0.0221


In [17]:
carbon_data.to_csv('../data/processed/carbon_data_processed.csv', index=False)


In [18]:
from supabase import create_client, Client
from dotenv import load_dotenv
import os
load_dotenv()
_SUPABASE_URL = os.getenv("SUPABASE_URL")
_SUPABASE_KEY = os.getenv("SUPABASE_KEY")

if not _SUPABASE_URL or not _SUPABASE_KEY:
    raise ValueError("Supabase URL 和 Key 未正确配置，请检查环境变量。")


_client: Client = create_client(_SUPABASE_URL, _SUPABASE_KEY)

def get_supabase_client() -> Client:
    return _client

def fetch_table(table_name: str):
    return _client.table(table_name).select("*").execute().data

def insert_data(table_name: str, data: dict):
    return _client.table(table_name).insert(data).execute().data

In [25]:
from supabase import create_client
import os

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

supabase = create_client(url, key)

# 使用自定义 SQL 查询所有表名
result = supabase.rpc(
    'sql',
    {
        'query': """
        SELECT table_name FROM information_schema.tables
        WHERE table_schema = 'public'
        AND table_type = 'BASE TABLE';
        """
    }
)
print(result.json)


{'query': "\n        SELECT table_name FROM information_schema.tables\n        WHERE table_schema = 'public'\n        AND table_type = 'BASE TABLE';\n        "}
